Here I will construct a CNN within tensorflow to identify the type of flower, won't actually be running this code here though becuase it requires a GPU, so that'll happen on a remote server but let's get the basics down here!


https://www.kaggle.com/alxmamaev/flowers-recognition

Should now only read in jpegs

Running cnn now, building/fixing architecture

In [1]:
%matplotlib inline


import cv2
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import tensorflow as tf
import os

tf.reset_default_graph()

/Users/noahkasmanoff/anaconda/lib/python3.6/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
#Making my daisy set in here

X_daisies = []
y_daisies = []
for file in os.listdir('/home/justinkterry/flowers/daisy'):
    if file.endswith(".jpg"):
        img = cv2.imread('//home/justinkterry/flowers/daisy/' + file)
        X_daisies.append((img))
        y_daisies.append([np.float64(1),np.float64(0),np.float64(0),np.float64(0),np.float64(0)])

        
        
#Now for dandelions
X_dandelions = []
y_dandelions = []
for file in os.listdir('/home/justinkterry/flowers/dandelion'):
    #print(filename)
    if file.endswith(".jpg"):
        img = cv2.imread('/home/justinkterry/flowers/dandelion/' + file)
        X_dandelions.append((img))
        y_dandelions.append([np.float64(0),np.float64(1),np.float64(0),np.float64(0),np.float64(0)])

#Now for roses
X_roses = []
y_roses = []
for file in os.listdir('/home/justinkterry/flowers/rose'):
    #print(filename)
    if file.endswith(".jpg"):
        img = cv2.imread('/home/justinkterry/flowers/rose/' + file)
        X_roses.append((img))
        y_roses.append([np.float64(0),np.float64(0),np.float64(1),np.float64(0),np.float64(0)])

#Now for sunflowers
X_sunflowers = []
y_sunflowers = []
for file in os.listdir('/home/justinkterry/flowers/sunflower'):
    #print(filename)
    if file.endswith(".jpg"):
        img = cv2.imread('/home/justinkterry/flowers/sunflower/' + file)
        X_sunflowers.append((img))
        y_sunflowers.append([np.float64(0),np.float64(0),np.float64(0),np.float64(1),np.float64(0)])

#Last but not least, tulips
X_tulips = []
y_tulips = []
for file in os.listdir('/home/justinkterry/flowers/tulip'):
    #print(filename)
    if file.endswith(".jpg"):
        img = cv2.imread('/home/justinkterry/flowers/tulip/' + file)
        X_tulips.append((img))
        y_tulips.append([np.float64(0),np.float64(0),np.float64(0),np.float64(0),np.float64(1)])

#Merging all at once. 4236 flowers in all. 

X_args = (X_daisies,X_dandelions,X_roses,X_sunflowers,X_tulips)
y_args = (y_daisies,y_dandelions,y_roses,y_sunflowers,y_tulips)
X_flowers = np.concatenate(X_args)
y_flowers = np.concatenate(y_args)



# Resize images
newsize = 360
def resize_with_pad(img, img_size):
    height, width, _ = img.shape
    ratio = img_size / max(height, width)
   # print(ratio)
    if ratio < 1:
        img = cv2.resize(img, (int(ratio * width), int(ratio * height)))
    padding = ((img_size - img.shape[0], 0), (img_size - img.shape[1],0), (0,0))

    return np.pad(img, padding, 'constant')

def rgb2gray(rgb):

    r, g, b = rgb[:,:,0], rgb[:,:,1], rgb[:,:,2]
    gray = 0.2989 * r + 0.5870 * g + 0.1140 * b

    return gray

for i in range(0,len(X_flowers)):
    
    X_flowers[i] = resize_with_pad(X_flowers[i], newsize)
    X_flowers[i] = rgb2gray(X_flowers[i])
   # X_flowers[i] = np.array(X_flowers[i]).reshape(1, 360,360,3) useless!
    #Extra line here reshapes to accept batch size as a dimension of accepted pics at once

for j in range(0,len(X_flowers)):
    X_flowers[j] = (X_flowers[j]).flatten()/255
  #  X_flowers[j] = np.array(X_flowers[j],dtype='float32')
    #Float 32 conversion
    


  #  y_flowers[k] = np.array(y_flowers[k],dtype='float32')
    #Change each element
    #Float 32 conversion

from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X_flowers, y_flowers, test_size=0.33, random_state=42)


def next_batch(num, data, labels):
    '''
    Return a total of `num` random samples and labels. 
    '''
    idx = np.arange(0 , len(data))
    np.random.shuffle(idx)
    idx = idx[:num]
    data_shuffle = [data[ i] for i in idx]
    labels_shuffle = [labels[ i] for i in idx]

    return np.asarray(data_shuffle), np.asarray(labels_shuffle)


#should convert vals to float 32, will get on that later.
# Training Parameters
learning_rate = .001
num_steps = 500
batch_size = 15
display_step = 10

# Network Parameters
num_inputs = newsize*newsize 
num_classes = 5 #total flower classes, daisy, etc. 5 total 
dropout = 0.5 # Dropout, probability to keep units

# tf Graph input
X = tf.placeholder(tf.float32, [None, num_inputs])
Y = tf.placeholder(tf.float32, [None, num_classes])
keep_prob = tf.placeholder(tf.float32) # dropout (keep probability)

# Create some wrappers for simplicity
def conv2d(x, W, b, strides=1):
    # Conv2D wrapper, with bias and relu activation
    x = tf.nn.conv2d(x, W, strides=[1, strides, strides, 1], padding='SAME')
    x = tf.nn.bias_add(x, b)
    return tf.nn.relu(x)


def maxpool2d(x, k=2):
    # MaxPool2D wrapper
    return tf.nn.max_pool(x, ksize=[1, k, k, 1], strides=[1, k, k, 1],
                          padding='SAME')


# Create model
def conv_net(x, weights, biases, dropout):
    # MNIST data input is a 1-D vector of 784 features (28*28 pixels)
    # Reshape to match picture format [Height x Width x Channel]
    # Tensor input become 4-D: [Batch Size, Height, Width, Channel]
    x = tf.reshape(x, shape=[-1, newsize, newsize, 1])

    # Convolution Layer
    conv1 = conv2d(x, weights['wc1'], biases['bc1'])
    # Max Pooling (down-sampling)
    conv1 = maxpool2d(conv1, k=2)

    # Convolution Layer
    conv2 = conv2d(conv1, weights['wc2'], biases['bc2'])
    # Max Pooling (down-sampling)
    conv2 = maxpool2d(conv2, k=2)

    # Fully connected layer
    # Reshape conv2 output to fit fully connected layer input
    fc1 = tf.reshape(conv2, [-1, weights['wd1'].get_shape().as_list()[0]])
    fc1 = tf.add(tf.matmul(fc1, weights['wd1']), biases['bd1'])
    fc1 = tf.nn.relu(fc1)
    # Apply Dropout
    fc1 = tf.nn.dropout(fc1, dropout)

    # Output, class prediction
    out = tf.add(tf.matmul(fc1, weights['out']), biases['out'])
    return out

# Store layers weight & bias
weights = {
    # 5x5 conv, 1 input, 32 outputs
    'wc1': tf.Variable(tf.random_normal([5, 5, 1, 32])),
    # 5x5 conv, 32 inputs, 64 outputs
    'wc2': tf.Variable(tf.random_normal([5, 5, 32, 64])),
    # fully connected, 7*7*64 inputs, 1024 outputs
    'wd1': tf.Variable(tf.random_normal([newsize/4*newsize/4*64, 1024])),
    # 1024 inputs, 10 outputs (class prediction)
    'out': tf.Variable(tf.random_normal([1024, num_classes]))
}

biases = {
    'bc1': tf.Variable(tf.random_normal([32])),
    'bc2': tf.Variable(tf.random_normal([64])),
    'bd1': tf.Variable(tf.random_normal([1024])),
    'out': tf.Variable(tf.random_normal([num_classes]))
}

# Construct model
logits = conv_net(X, weights, biases, keep_prob)
prediction = tf.nn.softmax(logits)

# Define loss and optimizer
loss_op = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(
    logits=logits, labels=Y))
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
train_op = optimizer.minimize(loss_op)


# Evaluate model
correct_pred = tf.equal(tf.argmax(prediction, 1), tf.argmax(Y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

# Initialize the variables (i.e. assign their default value)
init = tf.global_variables_initializer()

# Start training
with tf.Session() as sess:

    # Run the initializer
    sess.run(init)

    for step in range(1, num_steps+1):
        batch_x, batch_y = next_batch(batch_size,X_train,y_train)
        # Run optimization op (backprop)
        sess.run(train_op, feed_dict={X: batch_x,
                                      Y: batch_y,
                                      keep_prob: dropout})
        if step % display_step == 0 or step == 1:
            # Calculate batch loss and accuracy
            loss, acc = sess.run([loss_op, accuracy], feed_dict={X: batch_x,
                                                                 Y: batch_y,
                                                                 keep_prob: 1.0})
            print("Step " + str(step) + ", Minibatch Loss= " + \
                  "{:.4f}".format(loss) + ", Training Accuracy= " + \
                  "{:.3f}".format(acc))

    print("Optimization Finished!")

    # Calculate accuracy for 256 MNIST test images
    print("Testing Accuracy:", \
        sess.run(accuracy, feed_dict={X: X_test,
                                      Y: y_test,
                                      keep_prob: 1.0 }))

FileNotFoundError: [Errno 2] No such file or directory: '/home/justinkterry/flowers/daisy'

In [ ]:
14400.0/360

In [ ]:
Y

In [ ]:
feed_d

In [ ]:
np.shape(y_test)

In [ ]:
type(dropout)

In [ ]:
np.shape(X_train)

In [ ]:
len(X_train)

In [ ]:
X_train.reshape((len(X_train), np.shape(X_test[0])[0]))